In [1]:
import logging
logging.getLogger('tensorflow').disabled = True  # Biar ga banyak warning!
logging.getLogger('absl').disabled = True        # Biar ga banyak warning!

import config
import os, cv2
import time

import numpy as np
from tensorflow.keras.models import load_model

**Perhatikan batas memori yang tersedia, direkomendasikan hanya melakukan operasi < 30 gambar dalam 1 iterasi**

In [2]:
def predict(model_generator, images):
    t_in = time.time()
    
    for i in range(len(images)):
        images[i] = cv2.resize(images[i], (config.input_shape[1], config.input_shape[0]))
    
    images = np.array(images, 'float32')
    images = (images / 127.5) - 1
    images = model_generator(images).numpy()
    images = (images + 1) * 127.5
    image = np.array(images, 'uint8')
    
    t_out = time.time()
    duration = t_out - t_in
    
    return images, duration

def predict_folder(model_generator, source, target, batch_size=8):
    print('Generating color from %s'%(source), end='\r')
    file_names = os.listdir(source)
    num_batch = int( np.ceil(len(file_names)/batch_size) )
    for i in range(num_batch):
        images = []
        for j in range(i*batch_size, min((i+1)*batch_size, len(file_names))):
            image = cv2.imread( os.path.join(source, file_names[j]), 0 )
            images.append(image)
        images, _ = predict(model_generator, images)
        os.makedirs(target, exist_ok=True)
        for j in range(i*batch_size, min((i+1)*batch_size, len(file_names))):
            cv2.imwrite( os.path.join(target, file_names[j]), images[j-i*batch_size] )   
    print('Successfully generated color to %s'%(target))

In [3]:
model_name = 'v1-pix2pix'
model_generator = load_model( os.path.join(config.dir_model_saved, model_name, 'generator') )

In [4]:
predict_folder(model_generator, config.dir_test, config.dir_test_result)

Successfully generated color to ../Dataset/test-result/


#### Speed Test

In [5]:
def speed_test(model_generator, num):
    print('Testing for num=%d'%(num), end='\r')
    dummy = np.random.rand(num, config.input_shape[0], config.input_shape[1]) * 255
    dummy = np.array(dummy, 'uint8')
    _, duration = predict(model_generator, dummy)
    print('Result for num=%d is %.2f seconds (%.4f seconds per image)'%(num, duration, duration/num))

In [6]:
speed_test(model_generator, 1)
speed_test(model_generator, 10)
speed_test(model_generator, 50)
speed_test(model_generator, 100)

Result for num=1 is 0.65 seconds (0.6548 seconds per image)
Result for num=10 is 1.33 seconds (0.1329 seconds per image)
Result for num=50 is 4.06 seconds (0.0813 seconds per image)
Result for num=100 is 7.20 seconds (0.0720 seconds per image)
